<a href="https://colab.research.google.com/github/Ngueyap/Projet-HDH/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importer les bibliothèques nécessaires
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

# Créer une instance du moteur SQL
engine = create_engine('sqlite:///sante_data.db')

# Créer une session SQLAlchemy
Session = sessionmaker(bind=engine)
session = Session()

# Créer une classe de base pour les objets de la base de données
Base = declarative_base()


# Définir la classe pour la table "person"
class Person(Base):
    __tablename__ = 'person'

    person_id = Column(Integer, primary_key=True)
    person_source_value = Column(String)
    gender_concept_id = Column(Integer)
    year_of_birth = Column(Integer)  # Cette colonne sera remplie ultérieurement

# Créer la table dans la base de données
Base.metadata.create_all(engine)




<ipython-input-1-2bb82c6c71e2>:14: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [46]:
pd.read_csv('T_MCOaaC.csv', delimiter='|')


,NIR_ANO_17,RSA_NUM,ETA_NUM,COH_NAI_RET,COH_SEX_RET,DAT_RET,ENT_DAT,ETA_NUM_RET,EXE_SOI_AMD,EXE_SOI_AMF,...,SEJ_MER_RET,SEJ_NUM,SEJ_RET,SEX_RET,SOR_ANN,SOR_DAT,SOR_MOI,VID_HOSP_FOR,ENT_AM,SOR_AM
0,fiosftusfZACfuNhE,6766543446,250010162,True,False,c,5122013,True,198104,198104,...,True,31556,False,True,2014,1012014,6,418,199211,199211
1,fiosftusfZACfuNhE,1778813122,470015009,True,True,v,10032016,True,201310,201310,...,False,97756,False,True,1998,15042016,6,509,197208,197208
2,fiosftusfZACfuNhE,6079331430,30005540,False,True,Q,8021998,False,201106,201106,...,False,81628,False,False,2005,11031998,1,785,197205,197206
3,fiosftusfZACfuNhE,8205197405,750039182,False,False,V,18072010,False,201710,201711,...,False,69840,True,True,2001,18082010,2,410,201302,201302
4,fiosftusfZACfuNhE,8501373207,920003399,True,True,u,10111986,True,201208,201208,...,True,65673,False,False,1996,26111986,9,237,201211,201212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,xllHLbfuGaAjEBFSH,3247508060,130039738,False,False,r,21051973,True,199905,199905,...,True,62317,False,False,1961,23061973,10,151,199412,199501
538,MmWEyevQtWEtwYnZT,2696860019,450006002,True,False,b,28121981,False,200708,200709,...,False,74280,False,False,1986,14011982,4,746,198705,198705
539,MmWEyevQtWEtwYnZT,6260489159,290005362,True,False,K,10012000,False,199101,199101,...,False,98453,False,True,2004,11012000,11,850,199201,199202
540,HxbMiGPkIcNbbxgHD,9582689917,140032129,False,True,W,24051996,True,200807,200807,...,True,70413,False,False,2007,9071996,12,729,200303,200303


In [37]:
import pandas as pd

class DataCleaner:
    def __init__(self, file_path, delimiter):
        self.data = pd.read_csv(file_path, sep=delimiter)

    def get_num_rows_before_cleaning(self):
        return len(self.data)

    def remove_duplicates(self):
        self.data = self.data.drop_duplicates()

    def remove_missing_values(self):
        self.data = self.data.dropna()

    def get_cleaned_data(self):
        return self.data

    def get_distri_sexe(self):
        return self.data['COD_SEX'].value_counts()

    def map_sex_codes(self):
        # Remplacez les valeurs de COD_SEX selon votre codification
        sex_mapping = {1: 'Homme', 2: 'Femme', 0: 'Indéterminé', 9: 'Indéterminé'}
        self.data['COD_SEX'] = self.data['COD_SEX'].map(sex_mapping)



# Utilisation de la classe pour nettoyer deux fichiers CSV
if __name__ == '__main__':
    file_path1 = 'T_MCOaaC.csv'
    file_path2 = 'T_MCOaaB.csv'

    # Définissez le délimiteur pour chaque fichier
    delimiter2 = ','  # Délimiteur pour le premier fichier
    delimiter1 = '|'  # Délimiteur pour le second fichier

    # Créez les instances de DataCleaner en spécifiant les délimiteurs
    cleaner1 = DataCleaner(file_path1, delimiter1)
    cleaner2 = DataCleaner(file_path2, delimiter2)
    num_rows_before_cleaning1 = cleaner1.get_num_rows_before_cleaning()
    cleaner1.remove_duplicates()
    cleaner1.remove_missing_values()
    cleaned_data1 = cleaner1.get_cleaned_data()



    num_rows_before_cleaning2 = cleaner2.get_num_rows_before_cleaning()
    cleaner2.remove_duplicates()
    cleaner2.remove_missing_values()
    cleaned_data2 = cleaner2.get_cleaned_data()

    # Traitement de la colonne COD_SEX
    cleaner2.map_sex_codes()


    # nombres de lignes
    print(f"Nombre de lignes avant le nettoyage pour le fichier 1 : {num_rows_before_cleaning1}, "
          f"Nombre de lignes après le nettoyage : {len(cleaned_data1)}")
    print(f"Nombre de lignes avant le nettoyage pour le fichier 2 : {num_rows_before_cleaning2}, "
          f"Nombre de lignes après le nettoyage : {len(cleaned_data2)}")
    print("Distribution homme/femme dans T_MCOAAB :\n", cleaner2.get_distri_sexe())

Nombre de lignes avant le nettoyage pour le fichier 1 : 542, Nombre de lignes après le nettoyage : 514
Nombre de lignes avant le nettoyage pour le fichier 2 : 520, Nombre de lignes après le nettoyage : 513
Distribution homme/femme dans T_MCOAAB :
 Femme          190
Homme          165
Indéterminé    158
Name: COD_SEX, dtype: int64


In [49]:
# Insérez les valeurs de la colonne NIR_ANO_17 dans la table person
for index, row in cleaned_data1.iterrows():
    nir_ano_17 = row['NIR_ANO_17']

    # Créez une nouvelle instance de Person pour chaque valeur NIR_ANO_17
    person = Person(person_source_value=nir_ano_17)

    # Ajoutez cette instance à la session
    session.add(person)

# Validez les insertions
session.commit()